In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sys
sys.path.append("..")
import minimise
import energy_utils as energy
# Enable or disable Tensor Float 32 Execution
tf.config.experimental.enable_tensor_float_32_execution(False)
import matplotlib.pyplot as plt
import scipy.constants as const


In [ ]:
modelc1_path = "../../models/c1_dipole_Mar9.keras" 
modeln1_path = "../../models/n1_dipole_Mar9.keras"


model_c1 = keras.models.load_model(modelc1_path)
model_n1 = keras.models.load_model(modeln1_path)

In [ ]:
temp = 500 
L = 20
dx = 0.02  # spatial step in Angstroms
zbins = np.arange(0, L+dx, dx)

kappa_inv = 4.5  # inverse screening length

dielectric = 1.0
mu_correction = 0.0
Vext = np.zeros_like(zbins)
mu = -6.5 # dipole
muloc = - Vext + mu 
initial_guess = np.zeros_like(zbins)*0.0 + dx

In [ ]:


k = 1
 
amp = 0.5
    
A  = 2*np.pi*k/L        
E_field = -np.sin(A*zbins)*amp
elec_grad = np.zeros_like(zbins) + E_field / (const.Boltzmann * temp / const.elementary_charge)
Phi_field = np.cos(A*zbins)*amp/A 
elec = np.zeros_like(zbins) + Phi_field / (const.Boltzmann * temp / const.elementary_charge)
  
zs, rho_DFT, n_DFT, elecR_DFT = minimise.minimise_LR(
        model_c1, model_n1, zbins, muloc,  elec, temp,
        kappa_inv, dielectric, mu_correction, dx=dx,
        initial_guess=initial_guess, tolerance=1.8e-4, tolerance_restr = 1e-4,
        maxiter=10000,
        plot=True, plot_every=300, print_every=300,
        symmetric=True, chargesymmetry=True)

rho_mean = np.mean(rho_DFT)
betaFid, betaFexrho, betaFexphi, betamuterm = energy.get_free_energy(temp, rho_DFT, elec, mu, dx, model_c1, model_n1)

angtrom3_to_m3 = 1e30
pa_to_atm = 101325


omega_per_area = (const.k * temp * (betaFid + betaFexrho + betamuterm) / (dx*(len(rho_DFT)-1))) * angtrom3_to_m3/pa_to_atm
